# Setup cells

#### Extract and transform routine (pull and process data from SCUBA)

In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
import config

# connect to database
database = config.database
user = config.user
password = config.password
host = config.host
port = config.port

connection = pg.connect(database=database,
                        user=user,
                        password=password,
                        host=host,
                        port=port
                        )

anonymityDict = config.anonymityDict

def extract_and_transform(agencyID, where_clause):

    query = """
    WITH cte_1 AS (
        SELECT geoid10, cust_loc_hhsize, cust_loc_irr_area_sf, utility_name, cust_loc_id
        FROM cust_loc, census_block_polygons_2010 
        WHERE   ST_Within(cust_loc.geom,
                ST_setSRID(census_block_polygons_2010.geom,4326))
        AND (cust_loc_class = 'RESIDENTIAL_SINGLE'
        OR cust_loc_class = 'RESIDENTIAL_MULTI') 
        AND utility_name = '%s'
        %s),
    
    cte_2 AS (
        SELECT
        usage_ccf, usage_et_amount, make_date(usage.usage_year, usage.usage_month, '01') usage_date,
        cte_1.geoid10, cte_1.cust_loc_hhsize, cte_1.cust_loc_irr_area_sf, cte_1.utility_name, cte_1.cust_loc_id
        FROM cte_1
        JOIN usage
        ON cte_1.cust_loc_id = usage.cust_loc_id)
    
    SELECT
    geoid10,
    usage_date,
    SUM(cust_loc_hhsize) hhsize,
    SUM(cust_loc_irr_area_sf) irr_area_sf,
    SUM(usage_ccf) usage_ccf,
    AVG(usage_et_amount) usage_et_amount
    
    FROM
    cte_2
    GROUP BY
    geoid10, usage_date""" % (agencyID, where_clause)
    
    tempDF = psql.read_sql(query, connection)
    
    path = "./tempOut/"
    fname = "%s_census_block_usage.csv" % (anonymityDict[agencyID])
    tempDF.to_csv(path+fname)

#### Load routine (push processed data to CARTO)

In [2]:
import requests
import config

username = "california-data-collaborative"
apikey = config.apikey

def drop_old_table(agencyID):
    sql = "DROP TABLE %s_census_block_usage" % (anonymityDict[agencyID])
    url = "https://%s.carto.com/api/v2/sql/?q=%s&api_key=%s" % (username, sql, apikey)
    response = requests.get(url)
    
def load_new_table(agencyID):
    url = "https://%s.carto.com/api/v1/imports/?privacy=link&api_key=%s" % (username, apikey)
    
    path = "./tempOUT/"
    fname ="%s_census_block_usage.csv" % (anonymityDict[agencyID])
    f = {'file': open(path+fname)}
    
    response = requests.post(url, files=f)

---
# Execution cells

In [3]:
current_users = [{"agencyID":"MNWD", "where_clause":"AND geoid10 != '060590626431004'"}]

In [4]:
for user in current_users:
    extract_and_transform(user['agencyID'], user['where_clause'])
    drop_old_table(user['agencyID'])

Manually refresh account (to update account metadata)

In [ ]:
for user in current_users:
    load_new_table(user['agencyID'])

Manually refresh account (to update account metadata)